### **Bag of n_grams: Exercise**

- Fake news refers to misinformation or disinformation in the country which is spread through word of mouth and more recently through digital communication such as What's app messages, social media posts, etc.

- Fake news spreads faster than Real news and creates problems and fear among groups and in society.

- We are going to address these problems using classical NLP techniques and going to classify whether a given message/ text is **Real or Fake Message**.

- You will use a Bag of n-grams to pre-process the text and apply different classification algorithms.

- Sklearn CountVectorizer has the inbuilt implementations for Bag of Words.


### **About Data: Fake News Detection**

Credits: https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset


- This data consists of two columns.
        - Text
        - label
- Text is the statements or messages regarding a particular event/situation.

- label feature tells whether the given Text is Fake or Real.

- As there are only 2 classes, this problem comes under the **Binary Classification.**


In [27]:
#import pandas library
import pandas as pd

#read the dataset with name "Fake_Real_Data.csv" and store it in a variable df
df = pd.read_csv("Fake_Real_Data.csv")


#print the shape of dataframe
print(df.shape)

#print top 5 rows
df.head()

(9900, 2)


,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [28]:
#check the distribution of labels 
df.label.value_counts()

label
Fake    5000
Real    4900
Name: count, dtype: int64

In [29]:
min_samples = 4900
df_fake = df[df.label == "Fake"].sample(min_samples,random_state=505)
df_real = df[df.label=="Real"].sample(min_samples,random_state=505)
df = pd.concat([df_fake,df_real],axis=0)
df.label.value_counts()

#Add the new column "label_num" which gives a unique number to each of these labels 
df["label"] = df["label"].map({
    'Fake':0,
    "Real":1
    
})
#check the results with top 5 rows
df.head()

,Text,label
8225,Cher Reacts To Dr. Oz Reading Trump’s Health ...,0
4978,Putin Breaks Up With Trump In HUMILIATING Pub...,0
8864,WATCH: Texas Judge Leaves Republican Party In...,0
5700,Trump Just Committed Obstruction Of Justice B...,0
5588,Trump Humiliated By Being Forced To Sign Russ...,0


In [30]:
df.tail()

,Text,label
4123,Highlights: The Trump presidency on March 31 a...,1
2261,"Embroiled in controversies, Trump seeks boost ...",1
6982,"Subdued by Harvey, Congress reconvenes facing ...",1
8639,Senate panel approves Wray's nomination as FBI...,1
4586,"With Trump pick aboard, top U.S. court tackles...",1


### **Modelling without Pre-processing Text data**

In [32]:
#import train-test-split from sklearn 
from sklearn.model_selection import train_test_split

#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
X_train,X_test,y_train,y_test = train_test_split(
    df.Text,
    df.label,
    test_size=0.2,
    random_state=505,
    stratify=df.label
)


In [33]:
#print the shapes of X_train and X_test
print(X_train.shape)


(7840,)


**Attempt 1** :

1. using sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with unigram, bigram, and trigrams.
- use KNN as the classifier with n_neighbors of 10 and metric as 'euclidean' distance.
- print the classification report.


In [35]:
#1. create a pipeline object
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("NB",MultinomialNB())
])



#2. fit with X_train and y_train
clf.fit(X_train,y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.97      0.98      0.97       980
           1       0.98      0.97      0.97       980

    accuracy                           0.97      1960
   macro avg       0.97      0.97      0.97      1960
weighted avg       0.97      0.97      0.97      1960



**Attempt 2** :

1. using the sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with unigram, bigram, and trigrams.
- use **KNN** as the classifier with n_neighbors of 10 and metric as 'cosine' distance.
- print the classification report.


In [39]:
from sklearn.neighbors import KNeighborsClassifier

#1. create a pipeline object


clf = Pipeline([
	("vectorizer", CountVectorizer(ngram_range=(1, 2))),
	("knn", KNeighborsClassifier(n_neighbors=10, metric='cosine'))
])

#2. fit with X_train and y_train

clf.fit(X_train,y_train)


#3. get the predictions for X_test and store it in y_pred

y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.74      0.84       980
           1       0.79      0.99      0.88       980

    accuracy                           0.86      1960
   macro avg       0.89      0.86      0.86      1960
weighted avg       0.89      0.86      0.86      1960



<h3>Use text pre-processing to remove stop words, punctuations and apply lemmatization </h3>

In [41]:
#use this utility function to get the preprocessed text data

import spacy

# load english language model and create nlp object from it
nlp = spacy.load("en_core_web_sm") 

def preprocess(text):
    # remove stop words and lemmatize the text
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens) 

In [ ]:
# create a new column "preprocessed_txt" and use the utility function above to get the clean data
# this will take some time, please be patient
# df["preprocessedText"] = df["Text"].apply(preprocess)
# df.head()


KeyboardInterrupt: 

**Build a model with pre processed text**

In [ ]:
#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
#Note: Make sure to use only the "preprocessed_txt" column for splitting
#import train-test-split from sklearn 
from sklearn.model_selection import train_test_split

#Do the 'train-test' splitting with test size of 20% with random state of 2022 and stratify sampling too
X_train,X_test,y_train,y_test = train_test_split(
    df.preprocessedText,
    df.label,
    test_size=0.2,
    random_state=505,
    stratify=df.label
)



**Let's check the scores with our best model till now**
- Random Forest

**Attempt1** :

1. using the sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with only trigrams.
- use **RandomForest** as the classifier.
- print the classification report.


In [ ]:
#1. create a pipeline object
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

clf = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("NB",MultinomialNB())
])



#2. fit with X_train and y_train
clf.fit(X_train,y_train)


#3. get the predictions for X_test and store it in y_pred
y_pred = clf.predict(X_test)


#4. print the classfication report
print(classification_report(y_test,y_pred))

**Attempt2** :

1. using the sklearn pipeline module create a classification pipeline to classify the Data.

**Note:**
- using CountVectorizer with unigram, Bigram, and trigrams.
- use **RandomForest** as the classifier.
- print the classification report.


In [13]:
#1. create a pipeline object




#2. fit with X_train and y_train



#3. get the predictions for X_test and store it in y_pred



#4. print the classfication report


In [14]:
#finally print the confusion matrix for the best model

